In [1]:
import keras
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
import os

from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold

from iceburger.io import parse_json_data,color_composite, smooth, denoise, grayscale
from iceburger.combined_model_train import (create_dataset, get_model_notebook, 
                                            combined_model, gen_flow_multi_inputs,
                                            train_model, gen_model_weights, train_models)
from  iceburger.ensemble import KFoldEnsembleKerasModel


Using TensorFlow backend.


In [2]:
PRJ = "/workspace/iceburger"
DATA = os.path.join(PRJ,"data/processed/train_valid_knn11_impute.json")
outpath="./"

In [3]:
y_train, X_b, X_images = create_dataset(DATA, True)

Denoising and reshaping
Gray 1 done
Gray 2 done
Gray 3 done


/opt/conda/lib/python3.6/site-packages/skimage/filters/_gaussian.py:94: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))


RGB done


In [4]:
y_train_full, y_val,X_b_full, X_b_val,X_images_full, X_images_val = train_test_split(y_train, X_b, X_images, random_state=687, train_size=0.9)

In [5]:
lr = 1e-4
decay = 1e-4
channels = 3
relu = "relu"

model, partial_model = get_model_notebook(lr, decay, channels, relu)

In [6]:
model_arch_path = "/workspace/iceburger/checkpoints/convnet_arch.json"

with open(model_arch_path, "w") as jsonfile:
    jsonfile.write(model.to_json())

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 75, 3)         12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 73, 73, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 64)        0         
__________

In [8]:
conv = KFoldEnsembleKerasModel(model_arch_path, n_splits=3, n_repeats=1)

In [9]:
lr = 1e-3
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.,
                             height_shift_range=0.,
                             channel_shift_range=0,
                             zoom_range=0.2,
                             rotation_range=10)
optimizer = SGD(lr=lr, momentum=0.9, nesterov=True)
metrics = ["accuracy"]
epochs = 3
batch_size = 32
steps_per_epoch = X_images_full.shape[0] * 0.8 / batch_size

In [10]:
conv.train_generator(X_images_full, y_train_full, datagen, batch_size=batch_size,
                     checkpoint_name="convnet_images", loss="binary_crossentropy",
                     optimizer=optimizer, metrics=metrics, epochs=epochs,
                     steps_per_epoch=steps_per_epoch, callbacks=[])

Epoch 1/3
37/36 [==============================] - 11s 288ms/step - loss: 0.7228 - acc: 0.5282 - val_loss: 0.6935 - val_acc: 0.4689

{'000': {'best_epoch': 2,
  'best_val_loss': 0.69003121585766802,
  'checkpoint_path': 'convnet_images_000.hdf5',
  'history': <keras.callbacks.History at 0x7f7ceb1f3320>,
  'idx_train': array([ 477,  480,  484,  485,  486,  487,  488,  489,  490,  491,  492,
          493,  494,  495,  496,  497,  498,  499,  500,  501,  502,  503,
          504,  505,  506,  507,  508,  509,  510,  511,  512,  513,  514,
          515,  516,  517,  518,  519,  520,  521,  522,  523,  524,  525,
          526,  527,  528,  529,  530,  531,  532,  533,  534,  535,  536,
          537,  538,  539,  540,  541,  542,  543,  544,  545,  546,  547,
          548,  549,  550,  551,  552,  553,  554,  555,  556,  557,  558,
          559,  560,  561,  562,  563,  564,  565,  566,  567,  568,  569,
          570,  571,  572,  573,  574,  575,  576,  577,  578,  579,  580,
          581,  582,  583,  584,  585,  586,  587,  588,  589,  590,  591,
          592,  593,  594,  595,  596,  597,  598,  599,  600,  6